In [4]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001-2d193952f0849bbc.parquet', 'test': 'data/test-00000-of-00001-fb3536c9c871f904.parquet'}
df = pd.read_parquet("hf://datasets/fazni/role-based-on-skills-2.0/" + splits["train"])

In [5]:
df.head()

,Role,text,label,__index_level_0__
0,Mobile App Developer,Java JavaScript Android Development PHP HTML S...,6,2480
1,Machine Learning Engineer,Python Programming Language SQL Machine Learni...,5,3525
2,Network Engineer,MySQL Shell Scripting Linux Ubuntu Windows Net...,7,2281
3,Business Analyst,Java Project Management Microsoft Office HTML ...,0,1293
4,DevOps,PostgreSQL Teamwork Microsoft Azure Amazon Web...,4,3178


In [6]:
df['text']

0       Java JavaScript Android Development PHP HTML S...
1       Python Programming Language SQL Machine Learni...
2       MySQL Shell Scripting Linux Ubuntu Windows Net...
3       Java Project Management Microsoft Office HTML ...
4       PostgreSQL Teamwork Microsoft Azure Amazon Web...
                              ...                        
3655    Java C Data Analysis Microsoft Office Leadersh...
3656    Java JavaScript PHP C HTML Programming MySQL C...
3657    SQL Shell Scripting Linux Agile Methodologies ...
3658    Java Python Programming Language Data Analysis...
3659                                                  ...
Name: text, Length: 3660, dtype: object

In [7]:
df.isnull().sum()

Role                 0
text                 0
label                0
__index_level_0__    0
dtype: int64

In [9]:
df['text'] = df['text'].apply(lambda x: str(x).split())

In [11]:
df.head()

,Role,text,label,__index_level_0__
0,Mobile App Developer,"[Java, JavaScript, Android, Development, PHP, ...",6,2480
1,Machine Learning Engineer,"[Python, Programming, Language, SQL, Machine, ...",5,3525
2,Network Engineer,"[MySQL, Shell, Scripting, Linux, Ubuntu, Windo...",7,2281
3,Business Analyst,"[Java, Project, Management, Microsoft, Office,...",0,1293
4,DevOps,"[PostgreSQL, Teamwork, Microsoft, Azure, Amazo...",4,3178


In [16]:
df['text'] = df['text'].astype(str)

In [22]:
from nltk.stem.porter import PorterStemmer

def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
    
ps = PorterStemmer()
df['text'].apply(stem)

0       ['java', 'javascript', 'android', 'development...
1       ['python', 'programming', 'language', 'sql', '...
2       ['mysql', 'shell', 'scripting', 'linux', 'ubun...
3       ['java', 'project', 'management', 'microsoft',...
4       ['postgresql', 'teamwork', 'microsoft', 'azure...
                              ...                        
3655    ['java', 'c', 'data', 'analysis', 'microsoft',...
3656    ['java', 'javascript', 'php', 'c', 'html', 'pr...
3657    ['sql', 'shell', 'scripting', 'linux', 'agile'...
3658    ['java', 'python', 'programming', 'language', ...
3659    ['bachelor', 'of', 'science', 'university', 'c...
Name: text, Length: 3660, dtype: object

In [111]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')
# vectors = cv.fit_transform(df['text']).toarray()

In [112]:
vectors = cv.fit_transform(df['text']).toarray()

In [113]:
[vectors[0]]

[array([0, 0, 0, ..., 0, 0, 0])]

In [114]:
cosine_similarity([vectors[0]], [vectors[1]])

array([[0.02227177]])

In [130]:
print(cv.get_feature_names_out())

['03' '04' '05' ... 'بكالوريوس' 'جامعة' 'جامعه']


In [133]:

sk_list = ['java', 'javascript', 'android']
vector = []
for feature in cv.get_feature_names_out():
    # Check if the feature is in the skill list
    if feature in sk_list:
        vector.append(1)  # Skill is present
    else:
        vector.append(0)  # Skill is absent

print(cosine_similarity(vectors, [vector]))

[[0.33333333]
 [0.        ]
 [0.        ]
 ...
 [0.        ]
 [0.07053456]
 [0.        ]]


In [47]:
def recommend(job_role):
    job_index = df[df['Role'] == job_role].index[0]
    distances = similarity[job_index]
    job_lists = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    recommendations = []
    for i in job_lists:
        recommendations.append({
            'Role' : df.iloc[i[0]].Role,
            'Description' : df.iloc[i[0]].text,
            'label' : df.iloc[i[0]].label,
            'label' : df.iloc[i[0]].__index_level_0__,
            
        })
        new_df = pd.DataFrame(recommendations)
    return new_df
        # print(df.iloc[i[0]].Role)

In [48]:
recommend('Mobile App Developer')

,Role,Description,label
0,Mobile App Developer,"['Java', 'Android', 'Development', 'PHP', 'Mob...",2754
1,Mobile App Developer,"['Java', 'JavaScript', 'Android', 'Development...",2690
2,Mobile App Developer,"['JavaScript', 'Android', 'Development', 'HTML...",2604
3,Mobile App Developer,"['Java', 'Android', 'Development', 'PHP', 'HTM...",2786
4,Mobile App Developer,"['Java', 'JavaScript', 'PHP', 'Cascading', 'St...",2740


In [140]:
import numpy as np
def recommend_by_skills(skill_list):
    vector = []
    for feature in cv.get_feature_names_out():
    # Check if the feature is in the skill list
        if feature in skill_list:
            vector.append(1)  # Skill is present
        else:
            vector.append(0) 
    similarity = cosine_similarity(vectors, [vector])
    job_lists = sorted(list(enumerate(similarity)), reverse=True, key=lambda x:x[1])[1:6]
    print(similarity)
    recommendations = []
    for i in job_lists:
        recommendations.append({
            'Role' : df.iloc[i[0]].Role,
            'Description' : df.iloc[i[0]].text,
            'label' : df.iloc[i[0]].label,
            'label' : df.iloc[i[0]].__index_level_0__,
            
        })
        new_df = pd.DataFrame(recommendations)
    return new_df

In [146]:
recommend_by_skills(['python', 'programming', 'Language', 'SQL', 'HTML',  'JavaScript'])

[[0.        ]
 [0.21821789]
 [0.        ]
 ...
 [0.        ]
 [0.25916053]
 [0.        ]]


,Role,Description,label
0,Software Engineer,"['Python', 'Programming', 'Language', 'python'...",3381
1,Data Engineer,"['Java', 'JavaScript', 'Python', 'Programming'...",4040
2,Software Engineer,"['Java', 'JavaScript', 'PHP', 'Cascading', 'St...",572
3,Data Engineer,"['Java', 'JavaScript', 'Python', 'Programming'...",4091
4,Data Engineer,"['Python', 'Programming', 'Language', 'HTML', ...",4001


In [ ]:
df[df['label'] =]